## Decoding Strategies

### Initialization

In [40]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/mT5-small-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/mT5-small-HunSum-1")

prompt = """
A magyar szövetség vasárnapi tájékoztatása szerint a Jablonec nad Nisouban rendezett kontinensbajnokság szombati zárónapján a felnőtteknél a csoportos számban a Balázs Péter, Cséplő Kelen, Várszegi Vendel, Szász Tamás alkotta kvartett végzett bronzérmesként.

A 17 dobogós helyezéssel a magyarok másodikak lettek az éremtáblázaton

Kiss Bence csapatvezető szerint sikeres Európa-bajnokságot zártak a búvárok. Kiemelte: kiegyensúlyozott volt a válogatott, és külön öröm, hogy a felnőttek mellett a juniorok is eredményesen szerepeltek, mivel az ő eredményük is beleszámít az éremtáblázatba.

Tájékozódási búvárúszásban Monk, csillag, öt bója, szlalom, paralel és csoportos számokban rendeznek versenyeket, ezekben külön-külön bajnokokat is avatnak.
"""

input_ids = tokenizer.encode(prompt, return_tensors="pt")

### Currently Used (Beam Search)

In [4]:
nlp = pipeline(model=model, task='summarization', tokenizer=tokenizer)
nlp(prompt,
    max_length=128,
    num_beams=5,
    length_penalty=2,
    no_repeat_ngram_size=2,
    encoder_no_repeat_ngram_size=3,
    early_stopping=True
)

[{'summary_text': 'A magyar férfi búvárválogatott bronzérmet nyert a csehországi Zágrábban zajló európai öttusa-világbajnokságon, amelynek utolsó állomása az első helyen végzett.'}]

### Greedy Search

Required parameters:
- num_beams = 0
- do_sample = False

In [15]:
output = model.generate(
    input_ids,
    num_beams=1,
    do_sample=False,
    max_length=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A magyar férfi búvárválogatott bronzérmet nyert a budapesti Európa-bajnokságon, amelyen ezzel elsőként érmes lett.


### Multinomial sampling


Required parameters:
- do_sample = True

In [41]:
output = model.generate(
    input_ids,
    num_beams=1,
    do_sample=True,
    max_length=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  Minden második helyen végzett a kétnapos búvárúszásban résztvevő magyar búvóválogatott Strasbourgban.


### Beam-search multinomial sampling


Required parameters:
- do_sample = True
- num_beams > 1

In [42]:
output = model.generate(
    input_ids,
    num_beams=5,
    do_sample=True,
    max_length=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A magyar férfi búvárválogatott bronzérmet nyert a budapesti Európa-bajnokság szombati zárónapján.


### Diverse beam search


Required parameters:
- num_beams > 1
- num_beam_groups > 1

In [45]:
output = model.generate(
    input_ids,
    num_beams=5,
    num_beam_groups=5,
    do_sample=False,
    max_new_tokens=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A magyar férfi búvárválogatott bronzérmet nyert a budapesti Európa-bajnokságon, amelyen ezzel elsőként érmes lett.


### Contrastive search


Required parameters:
- penalty_alpha
- top_k

Not supported by our Models.

In [36]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model =  AutoModelForCausalLM.from_pretrained("gpt2")

prompt = """
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
"""

input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [39]:
output = model.generate(
    input_ids,
    num_beams=1,
    do_sample=False,
    penalty_alpha=0.6,
    top_k=4,
    max_length=128
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generated_text:  
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent."

The researchers found that the unicorns were able to communicate with each other through their tongues.

"They were able to


### Sources:
- https://huggingface.co/docs/transformers/v4.27.2/en/generation_strategies
- https://huggingface.co/blog/how-to-generate
- https://arxiv.org/abs/2202.06417
- https://arxiv.org/pdf/1610.02424.pdf